Using only 10 rows of the training set (due to performance)
### without preprocessing
- runtime: 27.005s
- best parameter settings: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 500}
- train scores: TBD
- test scores: [mae] 12.814, [mse] 1662.049, [r2] -0.040

### with preprocessing 1
- preprocessing: standardized scaling
- runtime: 27.328
- best parameter settings: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 500}
- train scores: TBD
- test scores: [mae] 11.985, [mse] 1647.030, [r2] -0.031

### with preprocessing 2
- preprocessing: standardized scaling, PCA down to 100 dimensions
- runtime: 31.536s
- best parameter settings: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 500}
- train scores: TBD
- test scores: [mae] 15.412, [mse] 1605.100, [r2] -0.005

In [18]:
### Load and preprocess data

%run base.ipynb

train_path = 'data/blog_feedback/blogData_train.csv'
test_path = 'data/blog_feedback/blogData_test-2012.02.01.00_00.csv'

train = pd.read_csv(train_path, header=None, sep = ",")
test = pd.read_csv(test_path, header=None, sep=",")

train_data = train.iloc[:,:279]
train_target = train.iloc[:,280]
test_data = test.iloc[:,:279]
test_target = test.iloc[:,280]

train_data, test_data = scale_data(train_data, test_data)
train_data, test_data = my_pca(train_data, test_data, 100)

print(train_data.shape)
print(train_target.shape)

(52397, 100)
(52397,)


In [4]:
### Train models

# Linear Regression

# SVR

# Gradient Boosted Decision Tree
param_fix = {'learning_rate': 0.01, 'loss': 'ls'}
cv = 10
param_grid = {'n_estimators': (50, 100, 150, 200, 300, 400, 500), 'max_depth': (1, 2, 3, 4, 5), 'min_samples_split': (2,3,5)}

gbt = run_boosted_tree(train_data.iloc[:10,:], train_target[:10], test_data, test_target, param_fix, cv, param_grid)

GridSearch initializing...
GradientBoostedRegressor model in training...
GradientBoostedRegressor model selected and fitted in 31.850 s

Best parameters selected by GridSearch: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 500}


In [6]:
### Test models

# Linear Regression

# SVR

# Gradient Boosted Decision Tree
gbt_pred = gbt.predict(test_data)

# GBT Metrics
mae = metrics.mean_absolute_error(test_target, gbt_pred)
mse = metrics.mean_squared_error(test_target, gbt_pred)
r2 = metrics.r2_score(test_target, gbt_pred)

print("\nGBT Metrics: [mae] %.3f, [mse] %.3f, [r2] %.3f" % (mae, mse, r2))


Metrics: [mae] 14.795, [mse] 1582.852, [r2] 0.009


In [19]:
### AutoML

run_autoML(train_data.iloc[:10,:], train_target[:10], test_data, test_target)

AutoML estimator initializing...
AutoML estimator in training...


ValueError: operands could not be broadcast together with shapes (10,2) (8,) (10,2) 